In [10]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from brnet import brnet
import glob
from os.path import *
from scipy.io import savemat, loadmat
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

import time

In [11]:
def downsample(signal, sfreq, downsample_factor=10):
    #return signal
    from scipy.signal import decimate 
    if sfreq==5000:
        signal = decimate(signal, downsample_factor, axis=1)
    return signal
    

In [18]:
def training(ffs, window_sec = 4, kernel_list = [3], folder = None, up_weight = [0,0,0,0], iter_num = 10000, lr_sche = 'step'):
    #window_sec:epoch長度
    #kernel_list:BRNet kernel size
    #folder:訊號存入位子
    #up_weight
    #iter_num:iteration number
    #le_sche:learning rate scheduler
    
    for f in ffs:
        mat = loadmat(f)
        downsample_factor=10
        sfreq = mat['sfreq_open'].flatten()[0]
        EEG_open = downsample(mat['EEG_before_bcg_open'], sfreq, downsample_factor=downsample_factor)
        ECG_open = downsample(mat['ECG_open'], sfreq, downsample_factor=downsample_factor)


        sfreq = mat['sfreq_close'].flatten()[0]
        EEG_close = downsample(mat['EEG_before_bcg_close'], sfreq, downsample_factor=downsample_factor)
        ECG_close = downsample(mat['ECG_close'], sfreq, downsample_factor=downsample_factor)

        if sfreq == 5000:
            sfreq = sfreq / downsample_factor
        sfreq = int(sfreq)

        label = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1',
             'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz',
             'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6',
             'CP5', 'CP6', 'TP9', 'TP10', 'POz']

        if not isdir(f"filtered/{folder}"):
            os.makedirs(f"filtered/{folder}", mode=0o777)
        npz_f = basename(f).replace('combined.mat', 'brnet.npz')
        EEG_f_open = brnet.run(EEG_open, ECG_open,
                                 kernels=[3], pretrain=False, winsize_sec = window_sec, sfreq=sfreq, iter_num = iter_num, up_weight = up_weight, lr_sche = lr_sche)  #iter_num=10000
        EEG_f_close = brnet.run(EEG_close, ECG_close,
                                  kernels=[3], pretrain=False, winsize_sec = window_sec, sfreq=sfreq, iter_num = iter_num, up_weight = up_weight,  lr_sche = lr_sche)    #iter_num=10000

        minlen = min(EEG_f_open.shape[1], EEG_f_close.shape[1])

        EEG_f_open = EEG_f_open[:, :minlen]
        EEG_f_close = EEG_f_close[:, :minlen]

        np.savez_compressed(f'filtered/{folder}/{npz_f}',
                            open=EEG_f_open, close=EEG_f_close, ECG_open=ECG_open, ECG_close = ECG_close, label=label, sfreq=sfreq)

## Onecase

In [19]:
#onecase
ffs = glob.glob('data/*CYH*_combined.mat')

folder = 'onecase_result'
iter_num = 15000
window_sec = 2
up_weight = [0,0,0,0]
lr_sche = 'cosine'

training(ffs, folder = folder, window_sec = window_sec, up_weight = up_weight, iter_num = iter_num, lr_sche = lr_sche)

Loss 125.054, lr: 0.00001: 100% 15000/15000 [01:35<00:00, 157.39it/s]
Loss 206.640, lr: 0.00003: 100% 15000/15000 [01:33<00:00, 159.72it/s]
